# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import time
import numpy as np
from PIL import Image
from keras.applications import InceptionV3
from keras.models import Sequential, Model, load_model
from keras.layers import Input, LSTM, Bidirectional, TimeDistributed, Dense, GlobalAveragePooling2D
from keras.utils import to_categorical

In [ ]:
import random
random.seed(42)

# Load DataSet and Model

In [ ]:
X = np.load(r'/content/drive/My Drive/Model Files/Full_No6_NP_X_train.npy') #loading parts of dataset
y = np.load(r'/content/drive/My Drive/Model Files/Full_No6_NP_y_train.npy')

In [ ]:
num_classes = 94
# One-hot encode the entire sequences of labels
y_one_hot = np.zeros((y.shape[0], y.shape[1], num_classes), dtype=np.int32)
for i in range(y.shape[0]):
    y_one_hot[i] = to_categorical(y[i], num_classes=num_classes)

In [ ]:
print(X.shape) # (no. samples, 5, 299, 299, 3)

(1305, 5, 299, 299, 3)

In [ ]:
# use when wanting to retrain or model
combined_model = load_model(r'/content/drive/My Drive/Model Files/Fixed_CombinedModel_50e_300units.h5')

# Combined Architecture

In [ ]:
num_classes = 94 # 93 unique words and 1 dummy

input_shape = (5, 299, 299, 3)
sequence_input = Input(shape=input_shape, batch_size=1) # batch_size must be fixed for tflite

inceptionv3_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))
for layer in inceptionv3_model.layers:
    layer.trainable = False

features = TimeDistributed(inceptionv3_model)(sequence_input)
pooled_features = TimeDistributed(GlobalAveragePooling2D())(features)
lstm_output = LSTM(units=300, return_sequences=True)(pooled_features)
output = TimeDistributed(Dense(units=num_classes, activation='softmax'))(lstm_output)
combined_model = Model(inputs=sequence_input, outputs=output)

combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

combined_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(1, 5, 299, 299, 3)]     0         
                                                                 
 time_distributed_1 (TimeDi  (1, 5, 8, 8, 2048)        21802784  
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (1, 5, 2048)              0         
 stributed)                                                      
                                                                 
 lstm (LSTM)                 (1, 5, 300)               2818800   
                                                                 
 time_distributed_3 (TimeDi  (1, 5, 94)                28294     
 stributed)                                                      
                                                             

# Training Model

In [ ]:
# Train the model
batch_size = 32
epochs = 50
combined_model.fit(X, y_one_hot, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/50
33/33 [==============================] - 33s 609ms/step - loss: 0.7948 - accuracy: 0.7722 - val_loss: 3.3562 - val_accuracy: 0.3088
Epoch 2/50
33/33 [==============================] - 8s 259ms/step - loss: 0.5531 - accuracy: 0.8372 - val_loss: 2.9875 - val_accuracy: 0.3387
Epoch 3/50
33/33 [==============================] - 8s 259ms/step - loss: 0.4022 - accuracy: 0.8814 - val_loss: 3.0426 - val_accuracy: 0.3548
Epoch 4/50
33/33 [==============================] - 8s 259ms/step - loss: 0.2056 - accuracy: 0.9498 - val_loss: 3.2484 - val_accuracy: 0.3272
Epoch 5/50
33/33 [==============================] - 9s 261ms/step - loss: 0.1110 - accuracy: 0.9801 - val_loss: 3.3390 - val_accuracy: 0.3172
Epoch 6/50
33/33 [==============================] - 8s 258ms/step - loss: 0.0761 - accuracy: 0.9889 - val_loss: 3.0933 - val_accuracy: 0.3648
Epoch 7/50
33/33 [==============================] - 8s 258ms/step - loss: 0.0522 - accuracy: 0.9954 - val_loss: 3.2286 - val_accuracy: 0.3571
Epoch

In [ ]:
combined_model.save(r'/content/drive/My Drive/Model Files/Fixed_CombinedModel_50e_300units.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Accuracy Checking + Test Loading

In [ ]:
train_results = combined_model.evaluate(X, y_one_hot)
print("Train Loss:", train_results[0])
print("Train Accuracy:", train_results[1])

41/41 [==============================] - 22s 301ms/step - loss: 0.6902 - accuracy: 0.8730
Train Loss: 0.6901681423187256
Train Accuracy: 0.8729501962661743


In [ ]:
X_test = np.load(r'/content/drive/My Drive/Model Files/Full_NP_X_test.npy')
y_test = np.load(r'/content/drive/My Drive/Model Files/Full_NP_y_test.npy')

In [ ]:
num_classes = 94
# One-hot encode the entire sequences of labels
y_test_ohe = np.zeros((y_test.shape[0], y_test.shape[1], num_classes), dtype=np.int32)
for i in range(y_test.shape[0]):
    y_test_ohe[i] = to_categorical(y_test[i], num_classes=num_classes)

In [ ]:
#combined_model = load_model(r'/content/drive/My Drive/Model Files/P1P2_CombinedModel_50e_300units.h5')

In [ ]:
test_results_1 = combined_model.evaluate(X_test[:933], y_test_ohe[:933]) #testing on [:933] then [933:] because of RAM
print("Test Loss:", test_results_1[0])
print("Test Accuracy:", test_results_1[1])

30/30 [==============================] - 18s 248ms/step - loss: 0.5751 - accuracy: 0.8322
Test Loss: 0.5750859975814819
Test Accuracy: 0.8321543335914612


In [ ]:
test_results_2 = combined_model.evaluate(X_test[933:], y_test_ohe[933:])
print("Test Loss:", test_results_2[0])
print("Test Accuracy:", test_results_2[1])

29/29 [==============================] - 8s 287ms/step - loss: 1.9210 - accuracy: 0.6370
Test Loss: 1.921007513999939
Test Accuracy: 0.6369637250900269


In [ ]:
# Update aggregated evaluation results
total_loss = (test_results_1[0] * X_test[:933].shape[0]) + (test_results_2[0] * X_test[933:].shape[0])
total_accuracy = (test_results_1[1] * X_test[:933].shape[0]) + (test_results_2[1] * X_test[933:].shape[0])
total_samples = X_test[:933].shape[0] + X_test[933:].shape[0]

# Calculate average loss and accuracy over the entire testing dataset
average_loss = total_loss / total_samples
average_accuracy = total_accuracy / total_samples

print("Average Loss:", average_loss)
print("Average Accuracy:", average_accuracy)

Average Loss: 1.2392785374427076
Average Accuracy: 0.735830629396128


# Inference Time Estimation

In [ ]:
inference_times = []

for i in range(933):
  input_data = X_test[-i] #change to i or -i
  input_data = np.expand_dims(input_data, axis=0)

  start_time = time.time()
  predictions = combined_model.predict(input_data)
  inference_time = time.time() - start_time
  inference_times.append(inference_time)

  print("Inference time: ", inference_time, " seconds")

average_inference_time = np.mean(inference_times)

print(average_inference_time, "seconds")